In [1]:
import torch
import os
import matplotlib.pyplot as plt
os.chdir("..")

from epilearn.models.SpatialTemporal.STGCN import STGCN
from epilearn.models.SpatialTemporal.MepoGNN import MepoGNN
from epilearn.models.SpatialTemporal.EpiGNN import EpiGNN
from epilearn.models.SpatialTemporal.DASTGN import DASTGN
from epilearn.models.SpatialTemporal.ColaGNN import ColaGNN
from epilearn.models.SpatialTemporal.EpiColaGNN import EpiColaGNN
from epilearn.models.SpatialTemporal.CNNRNN_Res import CNNRNN_Res
from epilearn.models.SpatialTemporal.ATMGNN import MPNN_LSTM, ATMGNN

from epilearn.models.Temporal.Dlinear import DlinearModel
from epilearn.models.Temporal.LSTM import LSTMModel
from epilearn.models.Temporal.GRU import GRUModel

from epilearn.data import UniversalDataset
from epilearn.utils import utils, transforms
from epilearn.tasks.forecast import Forecast

### Configs

In [2]:
# initial settings
device = torch.device('mps')
torch.manual_seed(7)

lookback = 12 # inputs size
horizon = 3 # predicts size

# permutation is True when using STGCN
permute = False

epochs = 10 # training epochs
batch_size = 50 # training batch size

### Initialize dataset

In [3]:
# load toy dataset
dataset = UniversalDataset()
dataset.load_toy_dataset()

### Initialize model and task
* prototype supports all models imported at the first cell

In [4]:
task = Forecast(prototype=EpiGNN, dataset=None, lookback=lookback, horizon=horizon)

### Add transformations to dataset

In [5]:
transformation = transforms.Compose({"features":[transforms.normalize_feat()], 
                                 'graph': [transforms.normalize_adj()], 
                                 'dynamic_graph': [transforms.normalize_adj()], 
                                 'states': []
                                 })
dataset.transforms = transformation

### Train Model
* for epicolagnn, loss='epi_cola' else loss='mse
* for STGCN, permute_dataset=True

In [6]:
config = None
result = task.train_model(dataset=dataset, config=config, loss='mse', epochs=5, permute_dataset=permute, device=device) # instead of config, we can also dircetly input some parameters

spatial-temporal model loaded!


100%|██████████| 5/5 [00:03<00:00,  1.54it/s]



Final Training loss: 9522.398098853326
Final Validation loss: 23821.14453125
Test MSE: 299950.25
Test MAE: 190.66786193847656
Test RMSE: 547.6771240234375


### Evaluate model

In [7]:
evaluation = task.evaluate_model()

Test MSE: 299950.25
Test MAE: 190.66786193847656
Test RMSE: 547.6771240234375


### Try more datasets

In [8]:
# load other datasets
datasets = [dataset]
raw_data = torch.load("datasets/covid_static.pt")
for name in ['Brazil', 'Austria', 'China']:
    data = raw_data[name]
    dataset = UniversalDataset()
    dataset.x = data['features']
    dataset.y = data['features'][:,:,0]
    dataset.graph = data['graph']
    dataset.states = data['features']
    dataset.dynamic_graph = None

    dataset.transforms = transformation
    datasets.append(dataset)


/var/folders/f_/9z5j7mn94d3bfnmns3bxbszr0000gn/T/ipykernel_12673/1224966375.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  raw_data = torch.load("datasets/covid_static.

In [9]:
for i, dataset in enumerate(datasets):
    print(f"dataset {i}")
    model = task.train_model(dataset=dataset, config=config, loss='mse', epochs=50, batch_size=50, permute_dataset=permute) # instead of config, we can also dircetly input some parameters

dataset 0
spatial-temporal model loaded!


100%|██████████| 50/50 [00:06<00:00,  7.67it/s]




Final Training loss: 8732.541434151786
Final Validation loss: 25009.900390625
Test MSE: 293662.21875
Test MAE: 187.66256713867188
Test RMSE: 541.9061279296875
dataset 1
spatial-temporal model loaded!


100%|██████████| 50/50 [00:02<00:00, 19.72it/s]




Final Training loss: 908052.6875
Final Validation loss: 601284.0625
Test MSE: 232661.734375
Test MAE: 259.1087341308594
Test RMSE: 482.3502197265625
dataset 2
spatial-temporal model loaded!


100%|██████████| 50/50 [00:02<00:00, 20.06it/s]




Final Training loss: 17236.51611328125
Final Validation loss: 703.6200561523438
Test MSE: 218.26226806640625
Test MAE: 9.065763473510742
Test RMSE: 14.773701667785645
dataset 3
spatial-temporal model loaded!


100%|██████████| 50/50 [00:01<00:00, 33.36it/s]




Final Training loss: 7147.6953125
Final Validation loss: 48518.01953125
Test MSE: 915881.4375
Test MAE: 107.65332794189453
Test RMSE: 957.0169677734375


### Try temporal models

In [10]:
task = Forecast(prototype=LSTMModel, dataset=None, lookback=lookback, horizon=horizon, device='cpu')
num_nodes = 47
mae_list=[]
rmse_list=[]
for region in range(num_nodes):
    print("region", region)
    result = task.train_model(dataset=datasets[-1], config=config, loss='mse', epochs=50, batch_size=50, region_idx=1, permute_dataset=False)
    mae_list.append(result['mae'])
    rmse_list.append(result['rmse'])

mae = torch.FloatTensor(mae_list)
rmse = torch.FloatTensor(rmse_list)
print(f"mae:{mae.mean()} {mae.std()}")
print(f"rmse:{rmse.mean()} {rmse.std()}")

region 0
temporal model loaded!


100%|██████████| 50/50 [00:00<00:00, 151.10it/s]




Final Training loss: 5.020066738128662
Final Validation loss: 9.171232223510742
Test MSE: 15.183773040771484
Test MAE: 2.93493390083313
Test RMSE: 3.8966360092163086
region 1
temporal model loaded!


100%|██████████| 50/50 [00:00<00:00, 165.08it/s]




Final Training loss: 4.789176940917969
Final Validation loss: 8.993369102478027
Test MSE: 14.404995918273926
Test MAE: 2.9321441650390625
Test RMSE: 3.795391321182251
region 2
temporal model loaded!


...



Final Training loss: 4.923417568206787
Final Validation loss: 9.143959045410156
Test MSE: 14.35190200805664
Test MAE: 2.943540334701538
Test RMSE: 3.7883903980255127
region 45
temporal model loaded!


100%|██████████| 50/50 [00:00<00:00, 157.56it/s]




Final Training loss: 4.7872209548950195
Final Validation loss: 9.052711486816406
Test MSE: 14.852538108825684
Test MAE: 2.9296348094940186
Test RMSE: 3.8538990020751953
region 46
temporal model loaded!


100%|██████████| 50/50 [00:00<00:00, 157.28it/s]



Final Training loss: 4.817765235900879
Final Validation loss: 9.118547439575195
Test MSE: 14.401469230651855
Test MAE: 2.9464633464813232
Test RMSE: 3.794926881790161
mae:2.9397404193878174 0.012147119268774986
rmse:3.858975410461426 0.052178412675857544
